In [1]:
import os
import sys
import numpy as np

import fiona

import rasterio as rio
from rasterio import features
from rasterio.merge import merge
from shapely.geometry import mapping
from shapely.geometry import shape

import shapely

from centerline.geometry import Centerline

In [2]:
DATADIR = "/home/lisa/roaddetection/data/"

MAP = os.path.join(DATADIR, 'train_raw/map')

The schema will be applied to the shapefile later.  

It needs to be **hard coded** and **should not be altered**.

In [3]:
schema= {
    'geometry': 'LineString', 
    'properties': {'id': 'int', 'name': 'int'}
}

The mapping dictionary defines the mapping between encodings used for paved / unpaved road pixels in the maps (255 / 127: unpaved / paved) to the encoding chosen in the original kmls (2 / 1: unpaved / paved).

In [4]:
map_px2rt = {255: 2, 127: 1}

Transform shapes extracted from one pixel map (output of U-Net) to a LineString object by using Centerline.  All hard-coded values applied in this function may be altered, however they were carefully selected by heuristic investigations.  

The transformed output is then saved as a shapefile and stored in out_path.  

In [5]:
def transform_raster(out_path, road_dict, crs, schema, mapper=map_px2rt):
    with fiona.open(out_path, 'w', 
                    crs=crs, 
                    driver='ESRI Shapefile', 
                    schema=schema) as output:
        for i, tile in enumerate(road_dict):
            poly = shape(tile['geometry'])
            newline = {}
            road_type = mapper[tile['pixel_value']]
            try:
                line = shapely.ops.linemerge(Centerline(poly, 10))
                line = line.simplify(7)

                newline['geometry'] = mapping(line)
                newline['properties'] = {'id': i+1, 'name': road_type}

                output.write(newline)
            except:
                pass   

Given a list of paths to (all) pixel maps *pixelmap_list* (U-Net output) the following functions associates each respective pixel map with its corresponding tile.
Pixel maps belonging to the same tile are then merged to produce a related shapefile which is named in accordance with its respective tile.  

Due to memory issues one might not be able to collect all pixel maps belonging to one tile into one huge related shapefile.  Therefore, a threshold is applied (default 500) to limit the maximum number of pixel maps that are collected and transformed for saving in a single shapefile.  
In this case, a suffix *i* will be appended to the name of the shapefile.  

In [10]:
def create_shapefiles(pixelmap_list, save_dir='shapes', threshold=500):
    tile_names = sorted(np.unique([l[:-9] for l in pixelmap_list]))
    for j, tile_name in enumerate(tile_names):
        pixelmap_selection = sorted([pixelmap for pixelmap in pixelmap_list if tile_name in pixelmap])
        chunks = (len(pixelmap_selection) // threshold) + 1
        for i in range(chunks):        
            road_map_collector = []
            for pixelmap_path in pixelmap_selection[i*threshold: (i+1)*threshold]:
                road_map = rio.open(pixelmap_path)
                out = road_map.meta.copy()
                crs = road_map.meta['crs']
                road_map_collector.append(road_map)
                
            merged_road_map, out_trans = merge(road_map_collector)  
            
            raod_map_dictionary = list({
                'geometry': s, 
                'pixel_value': int(v)}
                 for i, (s, v) in enumerate(
                        features.shapes(
                            merged_road_map[0, ...].astype(rio.uint8),
                            mask=None, 
                            connectivity=8,
                            transform=out_trans
                        )) if v > 0)

            
            file_name = '{}.shp'.format(tile_name.split('/')[-1])
            if chunks > 1: 
                file_name = '{}_{}.shp'.format(tile_name.split('/')[-1], str(i))
            file_path = os.path.join(save_dir, file_name)
            transform_raster(file_path, raod_map_dictionary, crs, schema)
            
            del merged_road_map, road_map_collector, raod_map_dictionary
        sys.stdout.write('Processed all pixel maps of tile {}.  Progress {} / {} tiles.'.\
                         format(tile_name.split('/')[-1], j+1, len(tile_names)))
        sys.stdout.flush()
    print('\nDone!')
    

Example how to use it.

Collect a list of pixel maps and then pass it to create_shapefiles.

Note that the the pixel map list can also be created by a pandas dataframe or in some other different way.

In [11]:
import glob

all_pixelmaps = sorted(glob.glob(MAP+'/*'))
print(all_pixelmaps)
            

['/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0000.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0001.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0002.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0003.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0004.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0005.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0006.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0007.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0008.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0009.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0010.tif', '/home/lisa/roaddetection/data/train_raw/map/20170815_005028_0c0b_3B_0011.tif', '/home/lisa/roaddetection/data/train_ra

In [ ]:
create_shapefiles(all_pixelmaps)